In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import networkx as nx
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
def get_common_elements(a, b): 
    # len(a) < len(b)
    jointlist = [] 
    for n in range(0, len(a)): 
        if a[n] in b: 
            jointlist.append(a[n])
            
    return jointlist

In [3]:
def isnull_sum(df):
    return df.isnull().sum() > 0

In [4]:
# Load HPRD gene list

In [5]:
gene_list = sio.loadmat("../data/HPRD_Adj/HPRD_Gene_list.mat")
hprd_genelist = [] 
for n in range(0, len(gene_list['gene_list'])): 
    hprd_genelist.append(gene_list['gene_list'][n][0][0])

In [6]:
# Load Huge ENSEMBL ID key

In [7]:
key_df = pd.read_excel('../data/ensembl-hugo-filtered.xlsx')

In [8]:
key_df.columns

Index(['Gene stable ID', 'Gene name'], dtype='object')

In [9]:
#Load data

In [10]:
rnadatafn = "../data/Expression Estimates - Gene Based_MMRF_CoMMpass_IA19_salmon_geneUnstrandedIgFiltered_tpm.tsv"
rnadatadf = pd.read_csv(rnadatafn, sep='\t')

In [11]:
cnadatafn = "../data/Copy Number Estimates_MMRF_CoMMpass_IA19_genome_gatk_cna_PerGene_LargestOverlap.tsv"
cnadatadf = pd.read_csv(cnadatafn, sep='\t')

In [14]:
np.max(np.max(cnadatadf).values[1:])

/Users/anish/opt/anaconda3/envs/myeloma/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


5.5822

In [15]:
np.min(np.min(cnadatadf).values[1:])

/Users/anish/opt/anaconda3/envs/myeloma/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


-30.9544

In [16]:
ensg_list = []
gene_list = [] 
hprd_gene_mask = np.zeros((len(hprd_genelist)))
found_genes = 0
missed_genes = 0 
found_eid = 0 


for n, gene_name in enumerate(hprd_genelist): 
    if np.sum(key_df['Gene name'].isin([gene_name])) > 0:
        df = key_df[key_df['Gene name'].isin([gene_name])]
        ensembl_id_list = df['Gene stable ID'].values
        
        # find overlap in rna genes        
        rna_hprd_overlap = rnadatadf[rnadatadf['Gene'].isin(ensembl_id_list)]['Gene']
        cna_hprd_overlap = cnadatadf[cnadatadf['Gene'].isin(ensembl_id_list)]['Gene']

        if len(cna_hprd_overlap) == 1 and len(rna_hprd_overlap) == 1: 
            found_eid = found_eid + 1 
            hprd_gene_mask[n] = 1
            ensg_list.append(cna_hprd_overlap.values[0])
            gene_list.append(gene_name)
           
        found_genes = found_genes + 1 
        
    else:
        missed_genes = missed_genes + 1
        
print("Found genes: ", found_genes)
print("Found EID: ", found_eid)
print("Missed genes: ", missed_genes)

Found genes:  8969
Found EID:  8903
Missed genes:  631


In [17]:
sum(hprd_gene_mask)

8903.0

In [18]:
cnadatadf = cnadatadf[cnadatadf['Gene'].isin(ensg_list)]

In [19]:
rnadatadf = rnadatadf[rnadatadf['Gene'].isin(ensg_list)]

In [20]:
cnadatadf.shape

(8903, 1094)

In [21]:
rnadatadf.shape

(8903, 929)

In [22]:
# Keep only 1st timepoint for the subjects

In [23]:
cna_subjectlist = cnadatadf.columns.values
rna_subjectlist = rnadatadf.columns.values

In [24]:
cna_subjectlist_filtered = [] 
for n in range(1, len(cna_subjectlist)): 
    if cna_subjectlist[n][10:14] == "1_BM": 
        cna_subjectlist_filtered.append(cna_subjectlist[n][0:9])

In [25]:
len(cna_subjectlist_filtered)

876

In [26]:
rna_subjectlist_filtered = [] 
for n in range(1, len(rna_subjectlist)): 
    if rna_subjectlist[n][10:14] == "1_BM": 
        rna_subjectlist_filtered.append(rna_subjectlist[n][0:9])

In [27]:
len(rna_subjectlist_filtered)

767

In [28]:
common_subject_ids = get_common_elements(rna_subjectlist_filtered, cna_subjectlist_filtered)

In [29]:
len(common_subject_ids)

669

In [30]:
cna_subject_list = [] 
cna_subject_list.append('Gene')
for subid in cna_subjectlist: 
    if subid[0:9] in common_subject_ids: 
        if subid[10:14] == "1_BM": 
            cna_subject_list.append(subid)

In [31]:
rna_subject_list = [] 
rna_subject_list.append('Gene')
for subid in rna_subjectlist: 
    if subid[0:9] in common_subject_ids: 
        if subid[10:14] == "1_BM": 
            rna_subject_list.append(subid)

In [32]:
rnadatadf = rnadatadf.reset_index()
cnadatadf = cnadatadf.reset_index()

rnadatadf = rnadatadf[rna_subject_list]
cnadatadf = cnadatadf[cna_subject_list]

In [33]:
# Turn all log2 values between -0.1604 and 0.1604 to 0

In [34]:
for subj in cnadatadf.columns: 
    if subj == 'Gene': 
        continue
    cnadatadf.loc[(cnadatadf[subj] >= -0.1604) & (cnadatadf[subj] <= 0.1604), subj] = 0 
    cnadatadf[subj] = cnadatadf[subj].apply(lambda x: np.power(2, x)*2)

In [35]:
for subj in rnadatadf.columns: 
    if subj == 'Gene': 
        continue
    rnadatadf[subj] = rnadatadf[subj].apply(lambda x: np.log2(x+1))

In [36]:
cnadatadf.min()

Gene                       ENSG00000000003
MMRF_1021_1_BM_CD138pos           0.980167
MMRF_1029_1_BM_CD138pos           1.018256
MMRF_1030_1_BM_CD138pos           1.027544
MMRF_1031_1_BM_CD138pos           0.303212
                                ...       
MMRF_2938_1_BM_CD138pos           1.008213
MMRF_2939_1_BM_CD138pos           0.718918
MMRF_2941_1_BM_CD138pos           1.012555
MMRF_2946_1_BM_CD138pos           0.934781
MMRF_2947_1_BM_CD138pos            0.98357
Length: 670, dtype: object

In [37]:
cnadatadf.shape

(8903, 670)

In [38]:
# Load HPRD adjacency matrix 

In [39]:
adj_mat = sio.loadmat("../data/HPRD_Adj/HPRD_Adj.mat")
adj_mat = adj_mat['adj']
adj_mat.shape

(9600, 9600)

In [40]:
np.sum(np.triu(adj_mat, 1) ) 

36822

In [41]:
75772/2

37886.0

In [42]:
# filter adjcency matrix to match genes that are in both IA 19 and HPRD

In [43]:
hprd_gene_mask = hprd_gene_mask.astype("bool")

In [44]:
filtered_adj_mat = adj_mat[hprd_gene_mask, :]
filtered_adj_mat = filtered_adj_mat[:, hprd_gene_mask]

In [45]:
filtered_adj_mat.shape

(8903, 8903)

In [46]:
type(filtered_adj_mat)

numpy.ndarray

In [47]:
G = nx.from_numpy_matrix(filtered_adj_mat)
largest_cc = max(nx.connected_components(G), key=len)
largest_cc = G.subgraph(largest_cc)
print(largest_cc.number_of_nodes())

8427


In [48]:
gene_list = np.array(gene_list)

In [49]:
ensg_list = np.array(ensg_list)

In [50]:
len(hprd_genelist)

9600

In [51]:
# get gene list 
nodes_to_keep = list(largest_cc.nodes)
nodes_to_keep = np.array(nodes_to_keep)

In [52]:
len(nodes_to_keep)

8427

In [53]:
# The gene list is alphabetical. the hprd is also alphabetical
# the rna cna data is not. 

In [54]:
def isInAlphabeticalOrder(word):
    for i in range(len(word) - 1):
        if word[i] > word[i + 1]:
            return False
    return True

In [55]:
isInAlphabeticalOrder(gene_list)

True

In [56]:
len(gene_list)

8903

In [57]:
ensg_list

array(['ENSG00000121410', 'ENSG00000148584', 'ENSG00000175899', ...,
       'ENSG00000162378', 'ENSG00000159840', 'ENSG00000074755'],
      dtype='<U15')

In [58]:
rnadatadf = rnadatadf.set_index('Gene').reindex(ensg_list).reset_index()

In [59]:
cnadatadf = cnadatadf.set_index('Gene').reindex(ensg_list).reset_index()

In [60]:
gene_list = gene_list[nodes_to_keep]

In [61]:
ensg_list = ensg_list[nodes_to_keep]

In [62]:
len(ensg_list)

8427

In [63]:
rnadatadf = rnadatadf.iloc[nodes_to_keep]
cnadatadf = cnadatadf.iloc[nodes_to_keep]

In [64]:
rnadatadf = rnadatadf.reset_index()
cnadatadf = cnadatadf.reset_index()

In [65]:
rnadatadf = rnadatadf[rna_subject_list]
cnadatadf = cnadatadf[cna_subject_list]

In [66]:
A = nx.to_numpy_array(largest_cc)

print(A.shape)

(8427, 8427)


In [67]:
A = np.triu(A, 1)

In [68]:
pd.DataFrame(A).to_csv("HPRD_ADJ_8427.csv")

In [69]:
data_tuples = list(zip(gene_list,ensg_list))
pd.DataFrame(data_tuples, columns=['Gene','ENSG']).to_csv("gene_key_8427.csv")

In [70]:
rnadatadf.shape

(8427, 670)

In [71]:
rnadatadf.to_csv("rna_8427_669.csv")

In [72]:
cnadatadf.to_csv("cna_8427_669.csv")

In [73]:
pd.DataFrame(cnadatadf.columns.values).to_csv("subject_list_669.csv")

In [74]:
import scipy

In [75]:
rna_data = rnadatadf.to_numpy()

In [76]:
cna_data = cnadatadf.to_numpy()

In [77]:
rna_data[:, 1:]

array([[0.06052349825273166, 0.07114949657275554, 0.01362493303702151,
        ..., 0.018246744974345898, 0.013382317100179682,
        0.09273461567681864],
       [2.6060939493622644, 3.318757898708065, 3.592433116737491, ...,
        0.7536507428545131, 1.3450670331493948, 1.211099438063769],
       [4.829616193672231, 4.890466148988911, 4.918977694489513, ...,
        4.870058885143275, 4.489170396840561, 5.319899683061693],
       ...,
       [3.536488130501283, 3.4373049860253775, 2.4827037482396057, ...,
        2.48767670667728, 2.906998312816846, 2.8202729260410155],
       [3.861340053494793, 4.2101160732806076, 2.5203216649738005, ...,
        2.7051994102170616, 2.9685571326859876, 0.43158659103884534],
       [4.116289891653365, 4.877979772770664, 3.8785095585802374, ...,
        5.616531200521562, 5.066145174552212, 4.772978275417309]],
      dtype=object)

In [78]:
data = {'ADJ':A, 'subject_list':cnadatadf.columns.values[1:], \
       'gene_list':gene_list, 'ensg_list':ensg_list, \
       'rna_data': rna_data[:, 1:], 'cna_data': cna_data[:, 1:]}

In [79]:
scipy.io.savemat('data_mmrf.mat', data)